In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import pickle

# Enable GPU logging for device placement
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Using GPU: ", tf.test.gpu_device_name())

def load_dataset(dataset_path, max_images_per_class=2000):
    image_paths = []
    labels = []
    class_mapping = {'Non-Glaucomous': 0, 'Glaucomous': 1}  # Update based on folder names

    for class_name, label in class_mapping.items():
        folder_path = os.path.join(dataset_path, class_name)
        if not os.path.exists(folder_path):
            print(f"Warning: Folder {class_name} not found at {folder_path}")
            continue

        all_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        all_files = sorted(all_files)[:max_images_per_class]

        for filename in all_files:
            image_path = os.path.join(folder_path, filename)
            image_paths.append(image_path)
            labels.append(label)

    print(f"Total images loaded: {len(image_paths)}")
    return image_paths, labels

def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized_image = cv2.resize(image_rgb, target_size, interpolation=cv2.INTER_AREA)
    normalized_image = resized_image.astype(np.float32) / 255.0
    return normalized_image

def prepare_data(image_paths, labels, target_size=(224, 224)):
    images = [preprocess_image(path, target_size) for path in image_paths]
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

def split_dataset(image_paths, labels, test_size=0.2, val_size=0.5):
    X_train, X_temp, y_train, y_temp = train_test_split(image_paths, labels, test_size=test_size, stratify=labels, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=val_size, stratify=y_temp, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test

def create_vgg16_model(input_shape=(224, 224, 3)):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Dataset path
dataset_path = "my_glaucoma_dataset"

# Define the path to save the model
model_save_path = "Models/my_glaucoma_dataset/vgg16_glaucoma_model.h5"

# Load dataset
image_paths, labels = load_dataset(dataset_path, max_images_per_class=2000)

# Split dataset
X_train_paths, X_val_paths, X_test_paths, y_train, y_val, y_test = split_dataset(image_paths, labels)

# Preprocess data
X_train, y_train = prepare_data(X_train_paths, y_train)
X_val, y_val = prepare_data(X_val_paths, y_val)
X_test, y_test = prepare_data(X_test_paths, y_test)

# Create and train VGG16 model
input_shape = (224, 224, 3)
vgg16_model = create_vgg16_model(input_shape)

print("\nTraining VGG16 model...")
history_vgg16 = vgg16_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=16,
    batch_size=8,
    verbose=1
)

# Save the model in TensorFlow format
print("\nSaving the TensorFlow model to Google Drive...")
vgg16_model.save(model_save_path)
print(f"TensorFlow model saved at {model_save_path}")

# Save the model as a .pkl file
pkl_model_save_path = "Model/vgg16_glaucoma_model.pkl"
print("\nSaving the model as a .pkl file...")
with open(pkl_model_save_path, 'wb') as f:
    pickle.dump(vgg16_model, f)
print(f"Pickle model saved at {pkl_model_save_path}")

# Evaluate the model on the test set
test_loss, test_accuracy = vgg16_model.evaluate(X_test, y_test, verbose=1)
print(f"\nTest Accuracy: {test_accuracy:.2f}")


Num GPUs Available:  0
Using GPU:  
Total images loaded: 0


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [7]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Path to the saved model
model_save_path = "Model/vgg16_glaucoma_model.h5"

# Load the saved model
vgg16_model = load_model(model_save_path)
print("Model loaded successfully!")

# Function to preprocess a single image
def preprocess_input_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found at path: {image_path}")
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized_image = cv2.resize(image_rgb, target_size, interpolation=cv2.INTER_AREA)
    normalized_image = resized_image.astype(np.float32) / 255.0
    return np.expand_dims(normalized_image, axis=0)  # Add batch dimension

# Function to make a prediction
def predict_image_class(image_path):
    # Preprocess the image
    preprocessed_image = preprocess_input_image(image_path)

    # Make prediction
    prediction = vgg16_model.predict(preprocessed_image)

    # Interpret the result
    class_mapping = {0: "Non-Glaucomous", 1: "Glaucomous"}
    predicted_class = int(np.round(prediction[0][0]))  # Convert probability to binary class
    confidence = prediction[0][0] * 100 if predicted_class == 1 else (1 - prediction[0][0]) * 100

    print(f"Prediction: {class_mapping[predicted_class]}")
    print(f"Confidence: {confidence:.2f}%")
    return class_mapping[predicted_class], confidence

# Example usage
image_path = "my_glaucoma_dataset/my_glaucoma_dataset/Non-Glaucomous/002_vflip.jpg"  # Replace with the path to your input image
predicted_class, confidence = predict_image_class(image_path)


Model loaded successfully!


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step
Prediction: Non-Glaucomous
Confidence: 99.83%
